In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/u

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-14 13:21:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.98MB/s    in 0.2s    

2023-02-14 13:21:53 (4.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [32]:
# Start a SparkSession
import findspark
findspark.init()

In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MobileCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Camera Reviews

In [34]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://amazonreviews011.s3.eu-west-2.amazonaws.com/amazon_reviews_us_Camera_v1_00.tsv" 
spark.sparkContext.addFile(url)
Camera_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv"), sep="\t", header=True)
# Show DataFrame
Camera_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [35]:
Camera_df.count()

1801974

In [36]:
df_camera = Camera_df.dropna()
df_camera.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [37]:
# Final count of records in df_camera
df_camera.count()

1801849

In [38]:
df_camera.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [49]:
# Create dataframe to match review_id table in PostgreSQL/RDS
review_id_df_camera = df_camera.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df_camera.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-08-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-08-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-08-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-08-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [50]:
review_id_df_camera = review_id_df_camera.dropDuplicates(["review_id"])

In [53]:
# Convert dataframe dtype to match review_id table schema dtypes
from pyspark.sql.types import * 

review_id_df_camera = review_id_df_camera.withColumn("customer_id",review_id_df_camera["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df_camera["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df_camera["review_date"].cast(DateType()))

review_id_df_camera.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [54]:
# Create dataframe to match products table in PostgreSQL/RDS
products_df_camera = df_camera.select(["product_id", "product_title"])
products_df_camera.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00I01JQJM|GoPro Rechargeabl...|
|B00TCO0ZAA|Professional 58mm...|
|B00B7733E0|Spy Tec Z12 Motio...|
|B006ZN4U34|Celestron UpClose...|
|B00HUEBGMU|Vidpro XM-L Wired...|
+----------+--------------------+
only showing top 5 rows



In [55]:
products_df_camera = products_df_camera.dropDuplicates(["product_id"])

In [56]:
products_df_camera.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [57]:
# Create dataframe to match customers table in PostgreSQL/RDS
customers_df_camera = df_camera.select(["customer_id"])
customers_df_camera.show(5)

+-----------+
|customer_id|
+-----------+
|    2975964|
|   23526356|
|   52764145|
|   47348933|
|   33680700|
+-----------+
only showing top 5 rows



In [58]:
customers_df_camera = customers_df_camera.groupBy("customer_id").count()
customers_df_camera.orderBy("customer_id").select(["customer_id", "count"])
customers_df_camera.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|     109519|    3|
|   45046643|    2|
|   45676353|   29|
|   49258595|    1|
|   11081770|    1|
+-----------+-----+
only showing top 5 rows



In [59]:
customers_df_camera = customers_df_camera.withColumnRenamed("count", "customer_count")
customers_df_camera.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     109519|             3|
|   45046643|             2|
|   45676353|            29|
|   49258595|             1|
|   11081770|             1|
+-----------+--------------+
only showing top 5 rows



In [60]:
customers_df_camera = customers_df_camera.withColumn("customer_id",customers_df_camera["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers_df_camera["customer_count"].cast(IntegerType()))

customers_df_camera.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [62]:
# Create dataframe to match vine table in PostgreSQL/RDS
vine_df_camera = df_camera.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df_camera.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [63]:
vine_df_camera = vine_df_camera.dropDuplicates(["review_id"])

In [65]:
vine_df_camera = vine_df_camera.withColumn("star_rating",vine_df_camera["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df_camera["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df_camera["total_votes"].cast(IntegerType()))

vine_df_camera.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Postgres Setup and writing to database

In [66]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cmmggpkmm8qx.eu-west-2.rds.amazonaws.com:5432/AmazonReview_Camera"
config = {"user":"root", 
          "password": "password1", 
          "driver":"org.postgresql.Driver"}

In [67]:
review_id_df_camera.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [68]:
products_df_camera.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [69]:
customers_df_camera.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [70]:
vine_df_camera.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)